# Options Volatility Screening Notebook

**Objective**: Identify large-cap stocks with attractive covered call income (<=50 DTE) while preserving upside via LEAP calls and limiting downside with protective puts.

**Strategy Framework**

1. **Sell volatility** with covered calls under 50 DTE.
2. **Capture upside** with a LEAP call.
3. **Protect downside** with a long/short-dated put near the Bollinger midline (20-day SMA).

**Data Source**: `yfinance` (Yahoo Finance). Data is suitable for research and screening, not institutional execution.


In [1]:
import os
import time
import warnings
from datetime import datetime
from typing import Optional

import numpy as np
import pandas as pd
import yfinance as yf
from yfinance import EquityQuery

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import display, Markdown

warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 140)

pio.templates.default = "plotly_white"

**Dependencies**: `yfinance`, `pandas`, `numpy`, `plotly`.
If needed: `python -m pip install yfinance pandas numpy plotly`


## Configuration

Adjust these parameters to reflect your screening and strategy preferences.


In [2]:
# Universe selection
USE_SCREEN = True
TICKER_OVERRIDE = []  # Used only if USE_SCREEN = False

SCREEN_PARAMS = dict(
    max_price=300.0,
    min_market_cap=2_000_000_000,
    min_roe=0.12,
    min_rev_growth=0.05,
    max_pe=40.0,
    max_ps=10.0,
    min_beta=1.0,
    min_inst_held=0.40,
    size=50,
    sort_by="eodvolume",
)

# Core strategy parameters
TARGET_DTES = [30, 50]  # Snapshot IV points
CC_TARGET_DTE = 50
CC_MAX_DTE = 60
LEAP_MIN_DTE = 365
LEAP_TARGET_MONEYNESS = 0.90  # 0.90 = 10% ITM
HEDGE_TARGET_DTE = 30
HEDGE_MAX_DTE = 50

OTM_LEVELS = [0.01, 0.03, 0.05, 0.07]  # 1%, 3%, 5%, 7% OTM
MAX_TERM_DTE = 180
TERM_STRUCTURE_SAMPLE = 1  # Use 1 for all, 2 for every other expiration

STRIKE_RANGE_PCT = 0.20  # For IV smile visualization
BOLLINGER_WINDOW = 20
HISTORY_PERIOD = "6mo"

RATE_LIMIT_SLEEP = 0.3
TERM_STRUCTURE_SLEEP_MULTIPLIER = 0.5

MAX_IV_SMILES = 6  # Limit plots to top N tickers by ATM IV (CC DTE)

# Quick-run overrides (set env QUICK_RUN=1 when executing)
QUICK_RUN = os.getenv("QUICK_RUN", "0") == "1"
DEFAULT_MAX_TICKERS = None
DEFAULT_TERM_STRUCTURE_SAMPLE = TERM_STRUCTURE_SAMPLE
DEFAULT_MAX_TERM_DTE = MAX_TERM_DTE

MAX_TICKERS = 8 if QUICK_RUN else DEFAULT_MAX_TICKERS
TERM_STRUCTURE_SAMPLE = 3 if QUICK_RUN else DEFAULT_TERM_STRUCTURE_SAMPLE
MAX_TERM_DTE = 90 if QUICK_RUN else DEFAULT_MAX_TERM_DTE

config_df = pd.DataFrame(
    {
        "Parameter": [
            "TARGET_DTES",
            "CC_TARGET_DTE",
            "LEAP_MIN_DTE",
            "LEAP_TARGET_MONEYNESS",
            "HEDGE_TARGET_DTE",
            "OTM_LEVELS",
            "MAX_TERM_DTE",
            "TERM_STRUCTURE_SAMPLE",
            "MAX_TICKERS",
        ],
        "Value": [
            str(TARGET_DTES),
            CC_TARGET_DTE,
            LEAP_MIN_DTE,
            LEAP_TARGET_MONEYNESS,
            HEDGE_TARGET_DTE,
            str(OTM_LEVELS),
            MAX_TERM_DTE,
            TERM_STRUCTURE_SAMPLE,
            MAX_TICKERS,
        ],
    }
)

display(config_df)

,Parameter,Value
0,TARGET_DTES,"[30, 50]"
1,CC_TARGET_DTE,50
2,LEAP_MIN_DTE,365
3,LEAP_TARGET_MONEYNESS,0.9
4,HEDGE_TARGET_DTE,30
5,OTM_LEVELS,"[0.01, 0.03, 0.05, 0.07]"
6,MAX_TERM_DTE,90
7,TERM_STRUCTURE_SAMPLE,3
8,MAX_TICKERS,8


## Core Functions


In [3]:
def screen_for_candidates(
    max_price: float = 300.0,
    min_market_cap: float = 2_000_000_000,
    min_roe: float = 0.12,
    min_rev_growth: float = 0.05,
    max_pe: float = 40.0,
    max_ps: float = 10.0,
    min_beta: float = 1.0,
    min_inst_held: float = 0.40,
    size: int = 50,
    sort_by: str = "eodvolume",
) -> list[str]:
    q = EquityQuery(
        "and",
        [
            EquityQuery("eq", ["region", "us"]),
            EquityQuery("is-in", ["exchange", "NMS", "NYQ"]),
            EquityQuery(
                "btwn", ["intradaymarketcap", min_market_cap, 4_000_000_000_000]
            ),
            EquityQuery("btwn", ["intradayprice", 10, max_price]),
            EquityQuery("btwn", ["peratio.lasttwelvemonths", 0, max_pe]),
            EquityQuery(
                "lt", ["lastclosemarketcaptotalrevenue.lasttwelvemonths", max_ps]
            ),
            EquityQuery("gte", ["returnontotalcapital.lasttwelvemonths", min_roe]),
            EquityQuery("gte", ["returnonequity.lasttwelvemonths", min_roe]),
            EquityQuery(
                "gte", ["totalrevenues1yrgrowth.lasttwelvemonths", min_rev_growth]
            ),
            EquityQuery("gte", ["pctheldinst", min_inst_held]),
            EquityQuery("gte", ["beta", min_beta]),
        ],
    )

    resp = yf.screen(q, size=size, sortField=sort_by, sortAsc=False)
    quotes = []
    if resp:
        if "quotes" in resp:
            quotes = resp.get("quotes", [])
        elif "finance" in resp:
            result = resp.get("finance", {}).get("result", [])
            if result:
                quotes = result[0].get("quotes", [])

    return [row.get("symbol") for row in quotes if row.get("symbol")]


def get_spot(ticker: str) -> Optional[float]:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        hist = t.history(period="1d")
        if not hist.empty and "Close" in hist.columns:
            return float(hist["Close"].iloc[-1])

        time.sleep(RATE_LIMIT_SLEEP)
        if hasattr(t, "fast_info") and t.fast_info:
            price = t.fast_info.get("lastPrice") or t.fast_info.get(
                "regularMarketPrice"
            )
            if price:
                return float(price)

        time.sleep(RATE_LIMIT_SLEEP)
        info = t.info
        if info:
            price = info.get("regularMarketPrice") or info.get("currentPrice")
            if price:
                return float(price)

        return None
    except Exception:
        return None


def get_expirations(ticker: str) -> list[tuple[str, int]]:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        exp_dates = t.options
        if not exp_dates:
            return []

        today = datetime.now().date()
        result = []
        for exp_str in exp_dates:
            try:
                exp_date = datetime.strptime(exp_str, "%Y-%m-%d").date()
                dte = (exp_date - today).days
                if dte > 0:
                    result.append((exp_str, dte))
            except ValueError:
                continue

        return sorted(result, key=lambda x: x[1])
    except Exception:
        return []


def find_expiration(
    expirations: list[tuple[str, int]],
    target_dte: Optional[int] = None,
    min_dte: Optional[int] = None,
    max_dte: Optional[int] = None,
) -> Optional[tuple[str, int]]:
    if not expirations:
        return None

    candidates = expirations
    if min_dte is not None:
        candidates = [e for e in candidates if e[1] >= min_dte]
    if max_dte is not None:
        candidates = [e for e in candidates if e[1] <= max_dte]
    if not candidates:
        return None

    if target_dte is None:
        return candidates[0]
    return min(candidates, key=lambda x: abs(x[1] - target_dte))


def add_chain_columns(
    df: pd.DataFrame, ticker: str, exp_date: str, spot: float
) -> pd.DataFrame:
    if df.empty:
        return df

    df = df.copy()
    exp_dt = datetime.strptime(exp_date, "%Y-%m-%d").date()
    dte = (exp_dt - datetime.now().date()).days

    df["ticker"] = ticker
    df["expiration"] = exp_date
    df["dte"] = dte
    df["spot"] = spot
    df["mid"] = (df["bid"] + df["ask"]) / 2
    df.loc[df["mid"] <= 0, "mid"] = df.get("lastPrice")
    df["moneyness"] = df["strike"] / spot

    return df


def fetch_chain(
    ticker: str, exp_date_str: str, spot: float
) -> tuple[pd.DataFrame, pd.DataFrame]:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        chain = t.option_chain(exp_date_str)

        calls = add_chain_columns(chain.calls, ticker, exp_date_str, spot)
        puts = add_chain_columns(chain.puts, ticker, exp_date_str, spot)
        return calls, puts
    except Exception:
        return pd.DataFrame(), pd.DataFrame()


def compute_atm_iv(
    calls: pd.DataFrame, puts: pd.DataFrame, spot: float
) -> Optional[float]:
    atm_ivs = []
    if not calls.empty and "impliedVolatility" in calls.columns:
        calls_valid = calls[
            calls["impliedVolatility"].notna() & (calls["impliedVolatility"] > 0)
        ]
        if not calls_valid.empty:
            idx = (calls_valid["strike"] - spot).abs().idxmin()
            atm_ivs.append(calls_valid.loc[idx, "impliedVolatility"])

    if not puts.empty and "impliedVolatility" in puts.columns:
        puts_valid = puts[
            puts["impliedVolatility"].notna() & (puts["impliedVolatility"] > 0)
        ]
        if not puts_valid.empty:
            idx = (puts_valid["strike"] - spot).abs().idxmin()
            atm_ivs.append(puts_valid.loc[idx, "impliedVolatility"])

    if atm_ivs:
        return float(np.mean(atm_ivs))
    return None


def compute_bollinger_midline(ticker: str, window: int = 20) -> Optional[float]:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        hist = t.history(period=HISTORY_PERIOD)
        if hist.empty or "Close" not in hist.columns:
            return None
        closes = hist["Close"].dropna()
        if len(closes) < window:
            return None
        return float(closes.rolling(window).mean().iloc[-1])
    except Exception:
        return None


def select_option_near_strike(
    df: pd.DataFrame,
    target_strike: float,
    direction: Optional[str] = None,
) -> Optional[pd.Series]:
    if df.empty:
        return None

    candidates = df[df["mid"].notna() & (df["mid"] > 0)]
    if candidates.empty:
        return None

    if direction == "below":
        candidates = candidates[candidates["strike"] <= target_strike]
    elif direction == "above":
        candidates = candidates[candidates["strike"] >= target_strike]

    if candidates.empty:
        return None

    idx = (candidates["strike"] - target_strike).abs().idxmin()
    return candidates.loc[idx]


def plot_iv_smile(
    ticker: str, calls: pd.DataFrame, puts: pd.DataFrame, spot: float, exp: str
) -> go.Figure:
    fig = go.Figure()

    low_strike = spot * (1 - STRIKE_RANGE_PCT)
    high_strike = spot * (1 + STRIKE_RANGE_PCT)

    if not calls.empty and "impliedVolatility" in calls.columns:
        calls_plot = calls[
            (calls["strike"] >= low_strike)
            & (calls["strike"] <= high_strike)
            & (calls["impliedVolatility"].notna())
            & (calls["impliedVolatility"] > 0)
        ]
        if not calls_plot.empty:
            fig.add_trace(
                go.Scatter(
                    x=calls_plot["strike"],
                    y=calls_plot["impliedVolatility"] * 100,
                    mode="lines+markers",
                    name="Calls",
                    marker=dict(size=6),
                    line=dict(width=2),
                    hovertemplate="Strike: $%{x:.2f}<br>IV: %{y:.2f}%<extra></extra>",
                )
            )

    if not puts.empty and "impliedVolatility" in puts.columns:
        puts_plot = puts[
            (puts["strike"] >= low_strike)
            & (puts["strike"] <= high_strike)
            & (puts["impliedVolatility"].notna())
            & (puts["impliedVolatility"] > 0)
        ]
        if not puts_plot.empty:
            fig.add_trace(
                go.Scatter(
                    x=puts_plot["strike"],
                    y=puts_plot["impliedVolatility"] * 100,
                    mode="lines+markers",
                    name="Puts",
                    marker=dict(size=6),
                    line=dict(width=2, dash="dash"),
                    hovertemplate="Strike: $%{x:.2f}<br>IV: %{y:.2f}%<extra></extra>",
                )
            )

    fig.add_vline(x=spot, line_dash="dash", line_color="green")
    fig.update_layout(
        title=f"{ticker} IV Smile (Exp: {exp})",
        xaxis_title="Strike Price ($)",
        yaxis_title="Implied Volatility (%)",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        height=450,
    )
    return fig

## Universe Selection


In [4]:
if USE_SCREEN:
    TICKERS = screen_for_candidates(**SCREEN_PARAMS)
else:
    TICKERS = list(TICKER_OVERRIDE)

if MAX_TICKERS:
    TICKERS = TICKERS[:MAX_TICKERS]

RUN_DATE = datetime.now().strftime("%Y-%m-%d")

display(Markdown(f"**Run Date**: {RUN_DATE}  "))
display(Markdown(f"**Quick Run**: {QUICK_RUN}"))
display(Markdown(f"**Tickers Loaded**: {len(TICKERS)}"))

display(pd.DataFrame({"Ticker": TICKERS}))

**Run Date**: 2026-02-02  

**Quick Run**: True

**Tickers Loaded**: 8

,Ticker
0,AAPL
1,F
2,CDE
3,AMZN
4,NFLX
5,FCX
6,ORCL
7,PATH


## Run Analysis


In [5]:
summary_rows = []
term_rows = []
cc_yield_rows = []
leap_rows = []
hedge_rows = []

chain_cache: dict[tuple[str, str], tuple[pd.DataFrame, pd.DataFrame]] = {}
chain_for_smile = {}


def get_chain_cached(
    ticker: str, exp_date: str, spot: float
) -> tuple[pd.DataFrame, pd.DataFrame]:
    key = (ticker, exp_date)
    if key not in chain_cache:
        chain_cache[key] = fetch_chain(ticker, exp_date, spot)
    return chain_cache[key]


for ticker in TICKERS:
    spot = get_spot(ticker)
    if spot is None:
        continue

    expirations = get_expirations(ticker)
    if not expirations:
        continue

    boll_mid = compute_bollinger_midline(ticker, window=BOLLINGER_WINDOW)

    row = {
        "ticker": ticker,
        "spot": spot,
        "boll_mid": boll_mid,
    }

    for target_dte in TARGET_DTES:
        picked = find_expiration(expirations, target_dte=target_dte)
        if not picked:
            continue

        exp_date, actual_dte = picked
        calls, puts = get_chain_cached(ticker, exp_date, spot)
        atm_iv = compute_atm_iv(calls, puts, spot)

        row[f"exp_{target_dte}"] = exp_date
        row[f"actual_dte_{target_dte}"] = actual_dte
        row[f"atm_iv_{target_dte}"] = atm_iv

    # Covered call chain (<= 50 DTE target)
    cc_pick = find_expiration(expirations, target_dte=CC_TARGET_DTE, max_dte=CC_MAX_DTE)
    if cc_pick:
        cc_exp, cc_dte = cc_pick
        cc_calls, cc_puts = get_chain_cached(ticker, cc_exp, spot)
        row["exp_cc"] = cc_exp
        row["dte_cc"] = cc_dte
        row["atm_iv_cc"] = compute_atm_iv(cc_calls, cc_puts, spot)

        chain_for_smile[ticker] = {
            "calls": cc_calls,
            "puts": cc_puts,
            "spot": spot,
            "exp": cc_exp,
        }

        if not cc_calls.empty:
            for otm in OTM_LEVELS:
                target_strike = spot * (1 + otm)
                selected = select_option_near_strike(cc_calls, target_strike)
                if selected is None:
                    continue

                premium_yield = selected["mid"] / spot
                annualized_yield = (
                    premium_yield * (365 / cc_dte) if cc_dte > 0 else None
                )

                cc_yield_rows.append(
                    {
                        "ticker": ticker,
                        "dte": cc_dte,
                        "target_otm": f"{otm:.0%}",
                        "target_strike": target_strike,
                        "actual_strike": selected["strike"],
                        "mid": selected["mid"],
                        "premium_yield": premium_yield,
                        "annualized_yield": annualized_yield,
                    }
                )

    # Term structure
    term_exps = [e for e in expirations if e[1] <= MAX_TERM_DTE]
    term_exps = term_exps[::TERM_STRUCTURE_SAMPLE]

    for exp_date, dte in term_exps:
        time.sleep(RATE_LIMIT_SLEEP * TERM_STRUCTURE_SLEEP_MULTIPLIER)
        calls, puts = get_chain_cached(ticker, exp_date, spot)
        atm_iv = compute_atm_iv(calls, puts, spot)
        if atm_iv is not None:
            term_rows.append(
                {
                    "ticker": ticker,
                    "expiration": exp_date,
                    "dte": dte,
                    "atm_iv": atm_iv,
                }
            )

    # LEAP call selection
    leap_pick = find_expiration(expirations, min_dte=LEAP_MIN_DTE)
    if leap_pick:
        leap_exp, leap_dte = leap_pick
        leap_calls, _ = get_chain_cached(ticker, leap_exp, spot)
        target_strike = spot * LEAP_TARGET_MONEYNESS
        leap_option = select_option_near_strike(leap_calls, target_strike)
        if leap_option is not None:
            leap_rows.append(
                {
                    "ticker": ticker,
                    "leap_exp": leap_exp,
                    "leap_dte": leap_dte,
                    "leap_strike": leap_option["strike"],
                    "leap_mid": leap_option["mid"],
                }
            )

    # Protective put near Bollinger midline
    if boll_mid is not None:
        hedge_pick = find_expiration(
            expirations, target_dte=HEDGE_TARGET_DTE, max_dte=HEDGE_MAX_DTE
        )
        if hedge_pick:
            hedge_exp, hedge_dte = hedge_pick
            _, hedge_puts = get_chain_cached(ticker, hedge_exp, spot)
            target_strike = min(boll_mid, spot)
            hedge_option = select_option_near_strike(
                hedge_puts, target_strike, direction="below"
            )
            if hedge_option is not None:
                hedge_rows.append(
                    {
                        "ticker": ticker,
                        "hedge_exp": hedge_exp,
                        "hedge_dte": hedge_dte,
                        "hedge_strike": hedge_option["strike"],
                        "hedge_mid": hedge_option["mid"],
                        "boll_mid": boll_mid,
                    }
                )

    summary_rows.append(row)

## Summary Dashboard


In [6]:
summary_df = pd.DataFrame(summary_rows)

if not summary_df.empty:
    # compute spot vs boll mid
    summary_df["spot_vs_boll_mid"] = np.where(
        summary_df["boll_mid"].notna(),
        (summary_df["spot"] / summary_df["boll_mid"]) - 1,
        np.nan,
    )

    display_cols = [
        "ticker",
        "spot",
        "exp_cc",
        "dte_cc",
        "atm_iv_cc",
        "spot_vs_boll_mid",
    ]

    for target_dte in TARGET_DTES:
        display_cols.extend(
            [f"exp_{target_dte}", f"actual_dte_{target_dte}", f"atm_iv_{target_dte}"]
        )

    summary_view = summary_df[display_cols].copy()

    summary_view = summary_view.rename(
        columns={
            "ticker": "Ticker",
            "spot": "Spot",
            "exp_cc": "CC Exp",
            "dte_cc": "CC DTE",
            "atm_iv_cc": "ATM IV (CC)",
            "spot_vs_boll_mid": "Spot vs Boll Mid",
        }
    )

    for target_dte in TARGET_DTES:
        summary_view = summary_view.rename(
            columns={
                f"exp_{target_dte}": f"Exp {target_dte}",
                f"actual_dte_{target_dte}": f"DTE {target_dte}",
                f"atm_iv_{target_dte}": f"ATM IV {target_dte}",
            }
        )

    # Build format dict explicitly to avoid nested unpacking issues
    format_dict = {
        "Spot": "${:,.2f}",
        "ATM IV (CC)": "{:.2%}",
        "Spot vs Boll Mid": "{:.2%}",
    }
    for dte in TARGET_DTES:
        format_dict[f"ATM IV {dte}"] = "{:.2%}"

    summary_style = summary_view.style.format(format_dict)

    display(summary_style.set_caption("Summary Metrics"))

    ranked = summary_df.dropna(subset=["atm_iv_cc"]).sort_values(
        "atm_iv_cc", ascending=False
    )

    if not ranked.empty:
        top = ranked.head(3)
        bottom = ranked.tail(3)

        takeaway_lines = ["**Key Takeaways**"]
        top_list = ", ".join([
            f"{row.ticker} ({row.atm_iv_cc:.2%})" for _, row in top.iterrows()
        ])
        bottom_list = ", ".join([
            f"{row.ticker} ({row.atm_iv_cc:.2%})" for _, row in bottom.iterrows()
        ])

        takeaway_lines.append(f"- Highest ATM IV (CC DTE): {top_list}")
        takeaway_lines.append(f"- Lowest ATM IV (CC DTE): {bottom_list}")

        display(Markdown("\n".join(takeaway_lines)))

,Ticker,Spot,CC Exp,CC DTE,ATM IV (CC),Spot vs Boll Mid,Exp 30,DTE 30,ATM IV 30,Exp 50,DTE 50,ATM IV 50
0,AAPL,$270.01,2026-03-20,46,24.13%,4.82%,2026-03-06,32,26.58%,2026-03-20,46,24.13%
1,F,$13.81,2026-03-20,46,32.03%,0.05%,2026-03-06,32,37.79%,2026-03-20,46,32.03%
2,CDE,$20.32,2026-03-20,46,90.92%,-8.55%,2026-03-06,32,94.63%,2026-03-20,46,90.92%
3,AMZN,$242.96,2026-03-20,46,38.38%,1.27%,2026-03-06,32,44.10%,2026-03-20,46,38.38%
4,NFLX,$82.76,2026-03-20,46,32.59%,-5.13%,2026-03-06,32,31.15%,2026-03-20,46,32.59%
5,FCX,$60.76,2026-03-20,46,50.23%,2.42%,2026-03-06,32,51.49%,2026-03-20,46,50.23%
6,ORCL,$160.06,2026-03-20,46,65.34%,-13.01%,2026-03-06,32,55.94%,2026-03-20,46,65.34%
7,PATH,$12.54,2026-03-20,46,80.52%,-17.06%,2026-03-06,32,60.06%,2026-03-20,46,80.52%


**Key Takeaways**
- Highest ATM IV (CC DTE): CDE (90.92%), PATH (80.52%), ORCL (65.34%)
- Lowest ATM IV (CC DTE): NFLX (32.59%), F (32.03%), AAPL (24.13%)

In [7]:
if not summary_df.empty:
    ranked = summary_df.dropna(subset=["atm_iv_cc"]).sort_values(
        "atm_iv_cc", ascending=False
    )

    if not ranked.empty:
        fig = px.bar(
            ranked,
            x="ticker",
            y="atm_iv_cc",
            title="ATM IV Ranking (Covered Call DTE)",
            labels={"ticker": "Ticker", "atm_iv_cc": "ATM IV"},
            text=ranked["atm_iv_cc"].map(lambda x: f"{x:.1%}"),
        )
        fig.update_traces(textposition="outside")
        fig.update_layout(yaxis_tickformat=".1%", height=450)
        fig.show()

term_df = pd.DataFrame(term_rows)
if not term_df.empty:
    fig = px.line(
        term_df,
        x="dte",
        y="atm_iv",
        color="ticker",
        markers=True,
        title="IV Term Structure (ATM IV vs DTE)",
        labels={"dte": "Days to Expiration", "atm_iv": "ATM IV"},
    )
    fig.update_layout(yaxis_tickformat=".1%", height=500)
    fig.show()

## Covered Call Income (<= 50 DTE)


In [8]:
cc_df = pd.DataFrame(cc_yield_rows)
if not cc_df.empty:
    display_cols = [
        "ticker",
        "dte",
        "target_otm",
        "actual_strike",
        "mid",
        "premium_yield",
        "annualized_yield",
    ]
    cc_view = cc_df[display_cols].copy()
    cc_view = cc_view.rename(
        columns={
            "ticker": "Ticker",
            "dte": "DTE",
            "target_otm": "Target OTM",
            "actual_strike": "Strike",
            "mid": "Mid",
            "premium_yield": "Yield",
            "annualized_yield": "Ann. Yield",
        }
    )

    cc_style = cc_view.style.format(
        {
            "Strike": "${:,.2f}",
            "Mid": "${:,.2f}",
            "Yield": "{:.2%}",
            "Ann. Yield": "{:.1%}",
        }
    )

    display(cc_style.set_caption("Covered Call Yield Table"))

    heatmap = cc_df.pivot_table(
        index="ticker",
        columns="target_otm",
        values="annualized_yield",
        aggfunc="mean",
    ).sort_index()

    fig = px.imshow(
        heatmap,
        text_auto=".1%",
        color_continuous_scale="RdYlGn",
        aspect="auto",
        title="Annualized Covered Call Yield Heatmap",
    )
    fig.update_layout(
        xaxis_title="OTM Level",
        yaxis_title="Ticker",
        coloraxis_colorbar=dict(title="Ann. Yield"),
        height=500,
    )
    fig.show()

,Ticker,DTE,Target OTM,Strike,Mid,Yield,Ann. Yield
0,AAPL,46,1%,$275.00,$6.50,2.41%,19.1%
1,AAPL,46,3%,$280.00,$4.65,1.72%,13.7%
2,AAPL,46,5%,$285.00,$3.22,1.19%,9.5%
3,AAPL,46,7%,$290.00,$2.08,0.77%,6.1%
4,F,46,1%,$14.00,$0.49,3.58%,28.4%
5,F,46,3%,$14.00,$0.49,3.58%,28.4%
6,F,46,5%,$14.85,$0.22,1.59%,12.6%
7,F,46,7%,$14.85,$0.22,1.59%,12.6%
8,CDE,46,1%,$20.00,$2.77,13.66%,108.4%
9,CDE,46,3%,$20.00,$2.77,13.66%,108.4%


## IV Smile (<= 50 DTE)


In [9]:
if not summary_df.empty:
    ranked = summary_df.dropna(subset=["atm_iv_cc"]).sort_values(
        "atm_iv_cc", ascending=False
    )
    tickers_to_plot = ranked["ticker"].head(MAX_IV_SMILES).tolist()

    for ticker in tickers_to_plot:
        chain = chain_for_smile.get(ticker)
        if not chain:
            continue
        fig = plot_iv_smile(
            ticker,
            chain["calls"],
            chain["puts"],
            chain["spot"],
            chain["exp"],
        )
        fig.show()

## LEAP and Protective Put Snapshot


In [10]:
leap_df = pd.DataFrame(leap_rows)
hedge_df = pd.DataFrame(hedge_rows)

if not leap_df.empty:
    leap_view = leap_df.rename(
        columns={
            "ticker": "Ticker",
            "leap_exp": "LEAP Exp",
            "leap_dte": "LEAP DTE",
            "leap_strike": "LEAP Strike",
            "leap_mid": "LEAP Mid",
        }
    )

    leap_style = leap_view.style.format(
        {
            "LEAP Strike": "${:,.2f}",
            "LEAP Mid": "${:,.2f}",
        }
    )
    display(leap_style.set_caption("LEAP Call Candidates"))

if not hedge_df.empty:
    hedge_view = hedge_df.rename(
        columns={
            "ticker": "Ticker",
            "hedge_exp": "Hedge Exp",
            "hedge_dte": "Hedge DTE",
            "hedge_strike": "Hedge Strike",
            "hedge_mid": "Hedge Mid",
            "boll_mid": "Boll Mid",
        }
    )

    hedge_style = hedge_view.style.format(
        {
            "Hedge Strike": "${:,.2f}",
            "Hedge Mid": "${:,.2f}",
            "Boll Mid": "${:,.2f}",
        }
    )
    display(hedge_style.set_caption("Protective Put Candidates"))

,Ticker,LEAP Exp,LEAP DTE,LEAP Strike,LEAP Mid
0,AAPL,2027-06-17,500,$240.00,$58.70
1,F,2027-06-17,500,$12.00,$2.92
2,CDE,2028-01-21,718,$17.50,$10.20
3,AMZN,2027-06-17,500,$220.00,$58.55
4,NFLX,2027-06-17,500,$74.00,$21.93
5,FCX,2027-06-17,500,$55.00,$16.88
6,ORCL,2027-03-19,410,$145.00,$46.02
7,PATH,2028-01-21,718,$12.00,$5.53


,Ticker,Hedge Exp,Hedge DTE,Hedge Strike,Hedge Mid,Boll Mid
0,AAPL,2026-03-06,32,$255.00,$1.66,$257.60
1,F,2026-03-06,32,$13.50,$0.40,$13.80
2,CDE,2026-03-06,32,$20.00,$2.12,$22.22
3,AMZN,2026-03-06,32,$235.00,$7.97,$239.91
4,NFLX,2026-03-06,32,$82.00,$2.54,$87.24
5,FCX,2026-03-06,32,$59.00,$2.54,$59.32
6,ORCL,2026-03-06,32,$160.00,$10.10,$183.99
7,PATH,2026-03-06,32,$12.50,$0.84,$15.12


## Notes and Limitations

- This notebook is for screening and research only. It is not investment advice.
- `yfinance` data can be delayed, incomplete, or inconsistent.
- Covered call yields use mid prices and simple annualization; execution and slippage are not modeled.
